In [142]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_excel('AKL單元冷凍系統操作條件(20060912_20091231).xlsx',index_col=0,skiprows=2)
for i in df.columns:
  df[i] = pd.to_numeric(df[i],errors='coerce')
df = df.iloc[1:,:]
df = df.dropna(axis=1,how='all').dropna(axis=0)
df.head()

,入口溫度,出口溫度,入口壓力,出口壓力,出口溫度.1,硫酸濃度
2006/09/12,3.1,58.4,0.3,4.8,32.1,90.94
2006/09/13,3.0,58.3,0.3,4.9,32.4,91.31
2006/09/14,3.0,58.5,0.3,4.9,32.5,91.15
2006/09/15,3.0,58.5,0.3,4.9,32.6,91.62
2006/09/16,3.0,58.8,0.2,4.8,32.0,91.02


In [143]:
df.max(),df.min()

(入口溫度      11.20
 出口溫度      69.50
 入口壓力       0.30
 出口壓力       5.20
 出口溫度.1    38.60
 硫酸濃度      96.79
 dtype: float64,
 入口溫度       1.50
 出口溫度      57.60
 入口壓力       0.10
 出口壓力       4.20
 出口溫度.1    27.80
 硫酸濃度      86.97
 dtype: float64)

In [144]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler,MinMaxScaler
真理 = 42
model = SVR(C=真理,gamma=真理)
ss = StandardScaler()
mm = MinMaxScaler()
x_col = ['入口溫度','出口溫度','入口壓力','出口壓力','出口溫度.1']
y_col = '硫酸濃度'
df_ss = df.copy()
df_ss[:] = ss.fit_transform(df_ss[:])
df_ss

,入口溫度,出口溫度,入口壓力,出口壓力,出口溫度.1,硫酸濃度
2006/09/12,-0.324150,-1.685517,1.183148,-1.140222,-0.929716,0.228589
2006/09/13,-0.424920,-1.746344,1.183148,-0.269326,-0.731651,0.551514
2006/09/14,-0.424920,-1.624690,1.183148,-0.269326,-0.665629,0.411870
2006/09/15,-0.424920,-1.624690,1.183148,-0.269326,-0.599607,0.822072
2006/09/16,-0.424920,-1.442210,-0.716248,-1.140222,-0.995738,0.298410
...,...,...,...,...,...,...
2009/12/27,0.784317,0.930033,-2.615644,-2.011118,-0.599607,1.773391
2009/12/28,0.784317,0.869206,-2.615644,-2.011118,-0.533585,1.293368
2009/12/29,0.885087,0.808379,-0.716248,-2.011118,-0.599607,0.813344
2009/12/30,0.582778,0.808379,-2.615644,-2.011118,-0.665629,0.394415


In [145]:
df_ss[y_col].max()

5.3342913169504715

In [146]:
x,y = df_ss[x_col],df_ss[y_col]
model.fit(x,y)
model.score(x,y)

0.9662602029400069

In [147]:
df_ss.loc[:,x_col[0]].min()

-1.9364657099597178

In [148]:
import optuna
import numpy as np
request = {} 
request['output'] = df_ss[y_col].max()
def objective(trial: optuna.Trial):
  Input_df = pd.DataFrame(
    index = [0],
    columns = x_col
    )
  for key in x_col:
    Input_df.loc[0,key] = trial.suggest_uniform(
      name = key,
      low = max(df_ss.loc[:,key].min(),0),
      high = df_ss.loc[:,key].max()
      )
  output_df = model.predict(Input_df)
  output = output_df[0]
  total_loss = np.sqrt((request['output']-output)**2) #L1 LOSS
  return 1/total_loss # 求最大值

study = optuna.create_study()
study.optimize(objective, n_trials=真理)

[I 2022-06-14 11:21:18,117] A new study created in memory with name: no-name-da7b6644-d3d8-4a2d-a4c9-e16607153680
[I 2022-06-14 11:21:18,125] Trial 0 finished with value: 0.19239311446114568 and parameters: {'入口溫度': 3.479210819034152, '出口溫度': 2.465103619000444, '入口壓力': 0.1724172642512474, '出口壓力': 2.296511680755735, '出口溫度.1': 3.075358928750179}. Best is trial 0 with value: 0.19239311446114568.
[I 2022-06-14 11:21:18,132] Trial 1 finished with value: 0.19239311446114568 and parameters: {'入口溫度': 1.424158977300304, '出口溫度': 1.9447918439858947, '入口壓力': 0.38862574264359573, '出口壓力': 0.7999177402085089, '出口溫度.1': 0.0052720990601784175}. Best is trial 0 with value: 0.19239311446114568.
[I 2022-06-14 11:21:18,137] Trial 2 finished with value: 0.19239311446114568 and parameters: {'入口溫度': 1.1579901020822232, '出口溫度': 4.921918167356803, '入口壓力': 0.4194041367471484, '出口壓力': 0.5403761185166523, '出口溫度.1': 1.672964834694614}. Best is trial 0 with value: 0.19239311446114568.
[I 2022-06-14 11:21:18,143] Tri

In [149]:
study.best_params

{'入口溫度': 3.479210819034152,
 '出口溫度': 2.465103619000444,
 '入口壓力': 0.1724172642512474,
 '出口壓力': 2.296511680755735,
 '出口溫度.1': 3.075358928750179}

In [150]:
model.predict(pd.DataFrame(study.best_params,index=[0]))[0]

0.13660010642665243

In [155]:
df_answer = pd.DataFrame()
for i in study.best_params:
    df_answer[i] = [study.best_params[i]]
df_answer[y_col] = model.predict(pd.DataFrame(study.best_params,index=[0]))[0]
print('r2_score:',model.score(x,y))
print('answer:')
df_answer

r2_score: 0.9662602029400069
answer:


,入口溫度,出口溫度,入口壓力,出口壓力,出口溫度.1,硫酸濃度
0,3.479211,2.465104,0.172417,2.296512,3.075359,0.1366
